In [1]:
from __future__ import print_function

from keras.datasets import mnist
from keras.optimizers import Adam, RMSprop
from keras.utils import np_utils
from keras import callbacks
from keras import objectives
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Input, Dense, Lambda, merge, Dropout
from keras.models import Model
from keras import backend as K
from keras.utils.visualize_util import plot
from IPython.display import Image

import os
import numpy as np
import pandas as pd
import six

import combo

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
df = pd.read_csv("data/impurity_SiGe.csv")
# df = pd.read_csv("data/impurity_SiSi.csv")
y = df["TC"].as_matrix().astype("float32")
X = df.iloc[:, 0:-1].as_matrix().astype("float32")

In [3]:
# メタパラメータの設定
max_epochs = 500  # 最大エポック数
batch_size = 100  #  ミニバッチサイズ
n_zdim = 2  # 隠れ層の次元
z_epsilon = 1.0 # 隠れ変数zの事前分布の分散パラメータ

In [4]:
index = np.random.permutation(np.arange(X.shape[0]))
Xtest = X[index[0:2000], :]
Xtrain = X [index[2000:], :]
ytest = y[index[0:2000]]

In [ ]:
# Sampling Function 
# Variational AutoEncoder の特徴 
# z_epsilon -> 0 であれば標準のautoencoder
def sampling(args):
    z_mean, z_sigma = args
    epsilon = K.random_normal(shape=(n_zdim,), mean=0., std=z_epsilon)
    return z_mean + z_sigma * epsilon

In [ ]:
# Encoder Layer p(z|x)
# 3層ネットワーク
x = Input(shape=(Xtrain.shape[1],)) # 入力層
hidden = Dense(1000, activation='relu')(x)  # 隠れ層
d_hidden = Dropout(0.5)(hidden)
hidden2 = Dense(200, activation='relu')(d_hidden)  # 隠れ層
d_hidden2 = Dropout(0.5)(hidden2)
hidden3 = Dense(200, activation='relu')(d_hidden2)  # 隠れ層
d_hidden3 = Dropout(0.5)(hidden3)
z_mean = Dense(n_zdim, activation='linear')(d_hidden3) # p(z|x)の平均
z_sigma = Dense(n_zdim, activation='linear')(d_hidden3) # p(z|x)の標準偏差
z = Lambda(sampling, output_shape=(n_zdim,))([z_mean, z_sigma]) # z 〜 p(z|x)

In [ ]:
# Decoder Layer p(x|z)
decoder_h = Dense(200, activation='relu')
decoder_mean = Dense(Xtrain.shape[1], activation='sigmoid')
h_decoded = decoder_h(z)
d_h_decoded = Dropout(0.5)(h_decoded)
x_decoded_mean = decoder_mean(d_h_decoded)

In [ ]:
# Variational AutoEncoder Model の定義
vae = Model(x, x_decoded_mean)

# Encoder Model の定義
encoder = Model(x,z)

# Decoder Model の定義
z_in = Input(shape=(n_zdim,))
_h_decoded = decoder_h(z_in)
_x_decoded_mean = decoder_mean(_h_decoded)
decoder= Model(input=z_in, output=_x_decoded_mean)

In [ ]:
def binary_crossentropy(y_true, y_pred):
    return K.sum(K.binary_crossentropy(y_pred, y_true), axis=-1)

In [ ]:
def vae_loss(x, x_decoded_mean):
    reconst_loss = K.mean(binary_crossentropy(x, x_decoded_mean),axis=-1)
    latent_loss =  - 0.5 * K.mean(K.sum(1 + K.log(K.square(z_sigma)) - K.square(z_mean) - K.square(z_sigma), axis=-1))
    return reconst_loss + latent_loss

In [ ]:
vae.compile(optimizer=RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0.), loss=vae_loss)

In [ ]:
# callback 関数設計
if not os.path.exists("checkpoint"):
    os.makedirs("checkpoint")
    
# Early stopping に関する
es_cb = EarlyStopping(monitor='val_loss', patience=50, verbose=0, mode='auto')
cp_cb = ModelCheckpoint(filepath = os.path.join("checkpoint",'vae{epoch:02d}.hdf5'), 
                                              monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
cbks = [cp_cb]

In [ ]:
history = vae.fit(Xtrain, Xtrain, nb_epoch=max_epochs, batch_size=batch_size, callbacks=cbks, shuffle = True,  validation_data=(Xtest, Xtest))


In [ ]:
# plot loss
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(range(1,max_epochs), loss[1:], marker='.', label='loss')
plt.plot(range(1,max_epochs), val_loss[1:], marker='.', label='val_loss')
plt.legend(loc='best', fontsize=10)
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(encoder.predict(X)[:, 0], encoder.predict(X)[:, 1], c=y)
plt.colorbar()
plt.grid()
#@plt.xlim((-10, ))
#plt.ylim((-5, 5))
plt.savefig("SiGe.eps")

In [ ]:
class simulator(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __call__(self, z):
        t = None
        for n in six.moves.range(X.shape[0]):
            if np.allclose(z, X[n, :]):
                t = y[n]        
        
        if t is None:
            t = -1000
            print("手持ちのデータにありませぬ")
        
        return t 

In [ ]:
sim = simulator(X, y)

In [ ]:
# random sampling 
n = 100
nbasis = 5000
index = np.random.permutation(six.moves.range(X.shape[0]))
train_X = X[index[0:n], :]
train_t = y[index[0:n]]

In [ ]:
predictor = combo.blm.predictor(policy.config)
train = combo.variable(encoder.predict(train_X), train_t)
predictor.fit(train, nbasis)

In [ ]:
def score(x, predictor, train):
    z = x.copy()
    z = z.reshape((1, -1))
    return combo.search.score.EI(predictor, train, combo.variable(X=z))[0]

In [ ]:
z = np.array([-1.0, 1.0])
score(z, predictor, train)

In [ ]:
import scipy.optimize
import copy 

min_fun = np.inf

for n in six.moves.range(5):
    params = (np.random.rand(2) -0.5) * 3
    res = scipy.optimize.minimize(score, params, args=(predictor, train))
    if res.fun < min_fun:
        opt_x = res.x

In [ ]:
recon = decoder.predict(opt_x.reshape((1, 2)))
# reconstract(reconstract >= 0.5)=1
# reconstract(reconstract < 0.5)=0

In [ ]:
# top 8 
recon[recon >= 0.5]=1
recon[recon < 0.5]=0

In [ ]:
sim(recon)

In [ ]:
opt_x